### Classic Machine Learning
This notebook implements a five-fold cross-validation model selection pipeline for classic machine learning algorithms. Once the best model has been selected, it is passed to a hyperparameter selection algorithm using Optuna. The tuned best model is then trained on the entire dataset with five buoys withheld for validation and evaluation of prediction accuracy.

Package imports

In [1]:
# Core Libraries
import gc
import glob
import math
import os
import time
from datetime import datetime, timedelta

# Data Handling
import netCDF4 as nc
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Geospatial Calculations
from geopy import Point
from geopy.distance import great_circle
from haversine import haversine
from scipy.spatial import cKDTree

# Machine Learning Models
from sklearn.ensemble import (
    GradientBoostingRegressor,
    RandomForestRegressor,
    VotingRegressor
)
from sklearn.linear_model import (
    BayesianRidge,
    ElasticNet,
    Lasso,
    LinearRegression,
    Ridge
)
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

# Model Evaluation and Optimization
import optuna
from optuna import create_study
from scipy.stats import randint, uniform
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score
)
from sklearn.model_selection import (
    GroupKFold,
    KFold,
    RandomizedSearchCV,
    cross_val_score,
    train_test_split
)

c:\Users\benem\anaconda3\envs\mlggeo2024_aobuoypredict\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Function to pre-process spatial data

In [2]:
# Precompute the KDTree and valid_time differences
def precompute_kdtree_and_time_diffs(uwnd_nc_file_path):
    try:
        print("Precomputing KDTree and time differences...")
        # Load the NetCDF file
        ds = nc.Dataset(uwnd_nc_file_path)

        # Extract the valid_time, latitudes, and longitudes from the NetCDF file
        valid_time = ds.variables['valid_time'][:]  # Assuming 'valid_time' is the variable name for time
        latitudes = ds.variables['latitude'][:]
        longitudes = ds.variables['longitude'][:]

        # Convert valid_time from seconds since 1970-01-01 to datetime
        base_time = datetime(1970, 1, 1)
        valid_time_dt = np.array([base_time + timedelta(seconds=int(ts)) for ts in valid_time], dtype='datetime64[ns]')

        # Create a KDTree for fast spatial lookup
        lat_lon_pairs = np.array([(lat, lon) for lat in latitudes for lon in longitudes])
        tree = cKDTree(lat_lon_pairs)

        print("KDTree and time differences precomputed successfully.")
        return tree, valid_time_dt, latitudes, longitudes, lat_lon_pairs
    except Exception as e:
        print(f"Error precomputing KDTree and time differences: {e}")
        raise

uwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_uwnd_2023.nc'
vwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_vwnd_2023.nc'
try:
    tree, valid_time_dt, latitudes, longitudes, lat_lon_pairs = precompute_kdtree_and_time_diffs(uwnd_nc_file_path)
except Exception as e:
    print(f"Error precomputing KDTree and time differences: {e}")
    raise

Precomputing KDTree and time differences...
KDTree and time differences precomputed successfully.


Function to extract wind components at a given lat/lon (preloads reanalysis netCDFs also)

In [3]:
uwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_uwnd_2023.nc'
vwnd_nc_file_path = '../data/raw/reanalyses/ERA5/era5_vwnd_2023.nc'

uwnd_ds = nc.Dataset(uwnd_nc_file_path)
vwnd_ds = nc.Dataset(vwnd_nc_file_path)

uwnd_array = uwnd_ds.variables['u'][:, 0, :, :]  # Assuming 'u' is the variable name for u-component wind and removing the pressure dimension
vwnd_array = vwnd_ds.variables['v'][:, 0, :, :]  # Assuming 'v' is the variable name for v-component wind and removing the pressure dimension

# Function to extract wind components
def extract_wind_components(lat, lon, dt, tree, valid_time_dt, latitudes, longitudes, lat_lon_pairs):
    try:
        # Convert the given datetime to a numpy datetime64 object
        row_datetime = np.datetime64(dt)

        # Find the value in the valid_time dimension closest in time to the datetime in the dataframe
        time_diffs = np.abs(valid_time_dt - row_datetime)
        closest_time_index = np.argmin(time_diffs)

        # Check if the calculated index is within the bounds of the uwnd_array
        if closest_time_index < 0 or closest_time_index >= uwnd_array.shape[0]:
            raise ValueError("The given datetime is out of bounds for the NetCDF data")

        # Select the corresponding netCDF slices
        uwnd_slice = uwnd_array[closest_time_index, :, :]
        vwnd_slice = vwnd_array[closest_time_index, :, :]

        # Find the grid cell of the netCDF slice closest to the given Latitude and Longitude position
        lat_lon = (lat, lon)
        _, closest_point_index = tree.query(lat_lon)
        closest_lat, closest_lon = lat_lon_pairs[closest_point_index]

        # Find the index of the closest latitude/longitude pair in the arrays
        lat_index = np.where(latitudes == closest_lat)[0][0]
        lon_index = np.where(longitudes == closest_lon)[0][0]

        # Extract the u and v wind components
        u_wind = uwnd_slice[lat_index, lon_index]
        v_wind = vwnd_slice[lat_index, lon_index]

        # Round wind components to 4 decimal places
        u_wind = round(u_wind, 4)
        v_wind = round(v_wind, 4)

        return u_wind, v_wind
    except Exception as e:
        print(f"Error extracting wind components: {e}")
        raise

Function to calculate new position from current position, displacement, and heading

In [4]:
# Import the math module
import math

# Redefine the calculate_new_position function with wrapping logic
def calculate_new_position(current_position, displacement, heading):
    R = 6371000  # Earth's radius in meters
    
    # Convert inputs to radians
    lat1 = math.radians(current_position[0])
    lon1 = math.radians(current_position[1])
    heading_rad = math.radians(heading)
    
    # Compute new latitude
    lat2 = math.asin(math.sin(lat1) * math.cos(displacement / R) +
                     math.cos(lat1) * math.sin(displacement / R) * math.cos(heading_rad))
    
    # Compute new longitude
    lon2 = lon1 + math.atan2(math.sin(heading_rad) * math.sin(displacement / R) * math.cos(lat1),
                             math.cos(displacement / R) - math.sin(lat1) * math.sin(lat2))
    
    # Convert back to degrees
    new_lat = math.degrees(lat2)
    new_lon = math.degrees(lon2)
    
    # Wrap longitude to [-180, 180]
    if new_lon > 180:
        new_lon -= 360
    elif new_lon < -180:
        new_lon += 360
    
    return new_lat, new_lon


## Model Optimization and Evaluation Functions

These functions handle model optimization with AutoML and evaluation of predictions. (implementation ongoing)

In [5]:
def evaluate_predictions(true_data, predicted_file):
    """
    Evaluate the accuracy of predictions against true data.
    """
    # Read predicted data
    pred_data = pd.read_csv(predicted_file)
    pred_data['Datetime'] = pd.to_datetime(pred_data['Datetime'])
    
    # Merge true and predicted data on datetime
    merged_data = pd.merge(
        true_data,
        pred_data,
        left_on=['datetime'],
        right_on=['Datetime'],
        suffixes=('_true', '_pred')
    )
    
    # Calculate position errors
    position_errors = []
    for _, row in merged_data.iterrows():
        true_pos = (row['Latitude_true'], row['Longitude_true'])
        pred_pos = (row['Latitude'], row['Longitude'])
        error_km = haversine(true_pos, pred_pos)
        position_errors.append(error_km)
    
    merged_data['position_error_km'] = position_errors
    
    # Calculate metrics
    metrics = {
        'mean_position_error_km': np.mean(position_errors),
        'median_position_error_km': np.median(position_errors),
        'max_position_error_km': np.max(position_errors),
        'std_position_error_km': np.std(position_errors),
        'rmse_lat': np.sqrt(mean_squared_error(merged_data['Latitude_true'], merged_data['Latitude'])),
        'rmse_lon': np.sqrt(mean_squared_error(merged_data['Longitude_true'], merged_data['Longitude'])),
        'mae_lat': mean_absolute_error(merged_data['Latitude_true'], merged_data['Latitude']),
        'mae_lon': mean_absolute_error(merged_data['Longitude_true'], merged_data['Longitude'])
    }
    
    return metrics, merged_data

def plot_trajectory_comparison(merged_data, buoy_id, model_name):
    """
    Plot true vs predicted trajectories
    """
    plt.figure(figsize=(12, 8))
    plt.plot(merged_data['Longitude_true'], merged_data['Latitude_true'], 
             'b-', label='True Trajectory')
    plt.plot(merged_data['Longitude'], merged_data['Latitude'], 
             'r--', label='Predicted Trajectory')
    plt.title(f'True vs Predicted Trajectory - Buoy {buoy_id}\nModel: {model_name}')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend()
    plt.grid(True)
    
    # Save plot
    plt.savefig(f'../data/processed/predictions/trajectory_comparison_{buoy_id}_{model_name}.png')
    plt.close()

def evaluate_all_predictions(val_data, predictions_dir):
    """
    Evaluate all prediction files in the specified directory
    """
    results = []
    prediction_files = glob.glob(f"{predictions_dir}/predicted_*.csv")
    
    for pred_file in prediction_files:
        # Extract buoy_id and model_name from filename
        filename = pred_file.split('/')[-1]
        buoy_id = filename.split('_')[1]
        model_name = filename.split('_')[2].replace('.csv', '')
        
        # Get true data for this buoy
        true_data_buoy = val_data[val_data['BuoyID'] == int(buoy_id)]
        
        # Calculate metrics
        metrics, merged_data = evaluate_predictions(true_data_buoy, pred_file)
        metrics['buoy_id'] = buoy_id
        metrics['model_name'] = model_name
        
        # Plot trajectory comparison
        plot_trajectory_comparison(merged_data, buoy_id, model_name)
        
        results.append(metrics)
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Save results
    results_df.to_csv(f'{predictions_dir}/evaluation_results.csv', index=False)
    
    return results_df

Iterative predictor function

In [6]:
def iterative_prediction(val_data, model, tree, valid_times, latitudes, longitudes, lat_lon_pairs):

    # Initialize an empty list to store predictions for all buoys
    all_predictions = []

    # Iterate over each unique BuoyID
    unique_buoy_ids = val_data['BuoyID'].unique()
    for buoy_id in unique_buoy_ids:
        buoy_data = val_data[val_data['BuoyID'] == buoy_id]

        # Initialize an empty list to store predictions for the current buoy
        predictions = []

        # Extract initial conditions for the current buoy
        current_lat, current_lon = buoy_data.iloc[0][['Latitude', 'Longitude']]
        current_uwnd, current_vwnd = buoy_data.iloc[0][['era5_uwnd', 'era5_vwnd']]

        # Add the initial condition as the first prediction
        predictions.append([current_lat, current_lon, buoy_data.iloc[0]['datetime']])

        for i in range(1, len(buoy_data)):
            next_row = buoy_data.iloc[i]

            # Prepare input data
            input_data = pd.DataFrame({
                'Latitude': [current_lat],
                'Longitude': [current_lon],
                'era5_uwnd': [current_uwnd],
                'era5_vwnd': [current_vwnd]
            })

            # Make prediction for displacement and heading
            predicted_displacement, predicted_heading = model.predict(input_data)[0]
            predicted_lat, predicted_lon = calculate_new_position(
                (current_lat, current_lon),
                predicted_displacement,
                predicted_heading
            )

            # Extract wind components at the predicted position and time
            predicted_wind_u, predicted_wind_v = extract_wind_components(
                predicted_lat, 
                predicted_lon, 
                next_row['datetime'],
                tree,
                valid_times,
                latitudes,
                longitudes,
                lat_lon_pairs
            )

            # Append the prediction for the current buoy
            predictions.append([predicted_lat, predicted_lon, next_row['datetime']])

            # Update current state for the next iteration
            current_lat, current_lon = predicted_lat, predicted_lon
            current_uwnd, current_vwnd = predicted_wind_u, predicted_wind_v

        # Append predictions of the current buoy to all_predictions
        all_predictions.extend(predictions)

    # Convert all predictions to a NumPy array before returning
    all_predictions_array = np.array(all_predictions, dtype=object)
    return all_predictions_array


Model selection, training, and validation (includes computation timing calculation)

NOTE: This script is extremely computationally intensive. Once the script has begun, interrupting it will crash the Python kernel and force you to re-run the entire notebook.

In [ ]:
# LightGBM verbosity suppression
lgb_params = {'verbose': -1}

# Load the data from the spreadsheet
buoy_data = pd.read_csv('../combined_buoy_data.csv')

# Drop unused columns
columns_to_keep = ['Latitude', 'Longitude', 'BuoyID', 'datetime', 'era5_uwnd', 'era5_vwnd', 'displacement', 'heading']
buoy_data = buoy_data[columns_to_keep].copy()
buoy_data['datetime'] = pd.to_datetime(buoy_data['datetime'])

# Define features and targets
X = buoy_data[['Latitude', 'Longitude', 'era5_uwnd', 'era5_vwnd', 'BuoyID', 'datetime']]
y = buoy_data[['displacement', 'heading']]
groups = buoy_data['BuoyID']

# Models to evaluate
model_configs = [
    ('ElasticNet', MultiOutputRegressor(ElasticNet(alpha=1.0, l1_ratio=0.5))),
    ('GradientBoosting', MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, max_depth=5))),
    ('RandomForest', RandomForestRegressor(n_estimators=100, max_depth=10)),
    ('XGBoost', MultiOutputRegressor(XGBRegressor(n_estimators=100, max_depth=6, objective='reg:squarederror'))),
    ('LightGBM', MultiOutputRegressor(lgb.LGBMRegressor(n_estimators=100, max_depth=6, **lgb_params)))
]

# GroupKFold for cross-validation
cv_folds = 5
group_kf = GroupKFold(n_splits=cv_folds)

# Ensure the predictions directory exists
predictions_dir = '../data/processed/predictions'
os.makedirs(predictions_dir, exist_ok=True)

# Initialize DataFrame to store results
results = []

# Cross-validation
for model_name, model in model_configs:
    print(f"\nTesting model: {model_name}")
    model_scores = []  # To store RMSE for each fold
    fold_times = []  # To store time taken for each fold

    for fold_num, (train_index, val_index) in enumerate(group_kf.split(X, y, groups=groups)):
        print(f"\nFold {fold_num + 1}")
        start_time = time.time()

        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Retain 'BuoyID' in X_val for iteration step
        X_val_with_buoyid = X_val.copy()
        X_train = X_train.drop(columns=['BuoyID', 'datetime'])
        X_val = X_val.drop(columns=['BuoyID', 'datetime'])

        # Train the model
        model.fit(X_train, y_train)

        # Predict iteratively
        y_pred = iterative_prediction(
            val_data=X_val_with_buoyid,
            model=model,
            tree=tree,
            valid_times=valid_time_dt,
            latitudes=latitudes,
            longitudes=longitudes,
            lat_lon_pairs=lat_lon_pairs
        )

        # Convert predictions to a DataFrame for easier handling
        y_pred = pd.DataFrame(y_pred, columns=['Latitude', 'Longitude', 'datetime'])

        # Exclude the datetime column for RMSE calculation and ensure numeric dtype
        y_pred_numeric = np.array(y_pred[['Latitude', 'Longitude']].to_numpy(), dtype=np.float64)

        # Ensure y_val is in the same format
        y_val_numeric = y_val.to_numpy()

        # Calculate RMSE
        try:
            rmse = np.sqrt(mean_squared_error(y_val_numeric, y_pred_numeric))
            model_scores.append(rmse)
            print(f"Fold {fold_num + 1} RMSE: {rmse:.3f}")
        except ValueError as e:
            print(f"Error calculating RMSE: {e}")
            continue

        # Record time taken for the fold
        fold_time = time.time() - start_time
        fold_times.append(fold_time)
        print(f"Fold {fold_num + 1} time: {fold_time:.2f} seconds")

        # Save predictions and true values to CSV
        predictions_df = pd.DataFrame({
            'BuoyID': X_val_with_buoyid['BuoyID'].values,  # Add BuoyID to the output
            'True Latitude': X_val_with_buoyid['Latitude'].values,  # Use latitude from X_val_with_buoyid
            'True Longitude': X_val_with_buoyid['Longitude'].values,  # Use longitude from X_val_with_buoyid
            'Predicted Latitude': np.round(y_pred_numeric[:, 0], 4),  # Predicted latitude rounded to 4 decimal places
            'Predicted Longitude': np.round(y_pred_numeric[:, 1], 4)  # Predicted longitude rounded to 4 decimal places
        })
        predictions_file = os.path.join(predictions_dir, f"{model_name}_fold{fold_num + 1}_predictions.csv")
        predictions_df.to_csv(predictions_file, index=False)

    # Store results for this model
    mean_rmse = np.mean(model_scores)
    std_rmse = np.std(model_scores)
    total_time = sum(fold_times)

    results.append({
        'Model': model_name,
        'Mean RMSE': mean_rmse,
        'RMSE StdDev': std_rmse,
        'Total Time (s)': total_time,
        'Mean Time per Fold (s)': np.mean(fold_times)
    })

    print(f"\nCompleted cross-validation for {model_name}. "
          f"Mean RMSE: {mean_rmse:.3f}, Std. Dev: {std_rmse:.3f}, Total Time: {total_time:.2f} seconds")

# Convert results to a DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('model_comparison_results.csv', index=False)

# Identify the best model based on mean RMSE
best_model_row = results_df.loc[results_df['Mean RMSE'].idxmin()]
print(f"\n=== Best model selected: {best_model_row['Model']} ===")
print(f"Mean RMSE: {best_model_row['Mean RMSE']:.3f}, Total Time: {best_model_row['Total Time (s)']:.2f} seconds")

# Store the best model
best_model = model_configs[results_df['Mean RMSE'].idxmin()][1]

print(f"Best model: {best_model_row['Model']}")
print(f"Mean RMSE: {best_model_row['Mean RMSE']:.3f}")
print(f"Total Time: {best_model_row['Total Time (s)']:.2f} seconds")


Testing model: ElasticNet

Fold 1
Fold 1 RMSE: 827.064
Fold 1 time: 380.83 seconds

Fold 2
Fold 2 RMSE: 767.791
Fold 2 time: 408.17 seconds

Fold 3
Fold 3 RMSE: 825.619
Fold 3 time: 413.79 seconds

Fold 4
Fold 4 RMSE: 813.975
Fold 4 time: 409.12 seconds

Fold 5
Fold 5 RMSE: 1829.535
Fold 5 time: 414.53 seconds

Completed cross-validation for ElasticNet. Mean RMSE: 1012.797, Std. Dev: 408.938, Total Time: 2026.44 seconds

Testing model: GradientBoosting

Fold 1
Fold 1 RMSE: 824.468
Fold 1 time: 984.29 seconds

Fold 2
Fold 2 RMSE: 768.219
Fold 2 time: 949.54 seconds

Fold 3
Fold 3 RMSE: 821.637
Fold 3 time: 927.72 seconds

Fold 4
Fold 4 RMSE: 818.564
Fold 4 time: 928.62 seconds

Fold 5
Fold 5 RMSE: 1830.465
Fold 5 time: 930.42 seconds

Completed cross-validation for GradientBoosting. Mean RMSE: 1012.671, Std. Dev: 409.423, Total Time: 4720.60 seconds

Testing model: RandomForest

Fold 1
Fold 1 RMSE: 824.421
Fold 1 time: 1031.53 seconds

Fold 2
Fold 2 RMSE: 766.104
Fold 2 time: 1012.37 s

Spoofing the previous result (necessary for resumation of project procedures in the case of kernel crashes, etc.)

In [7]:
import pandas as pd
import numpy as np

# Spoof the previous result
best_model_row = {
    'Model': 'XGBoost',
    'Mean RMSE': 1012.157,  
    'Total Time (s)': 5443.08 
}

# Print the spoofed result to ensure it's working as expected
print(f"Best model: {best_model_row['Model']}")
print(f"Mean RMSE: {best_model_row['Mean RMSE']}")
print(f"Total Time: {best_model_row['Total Time (s)']} seconds")

Best model: XGBoost
Mean RMSE: 1012.157
Total Time: 5443.08 seconds


Spoofing cont.

In [8]:
# LightGBM verbosity suppression
lgb_params = {'verbose': -1}

# Load the data from the spreadsheet
buoy_data = pd.read_csv('../combined_buoy_data.csv')

# Drop unused columns
columns_to_keep = ['Latitude', 'Longitude', 'BuoyID', 'datetime', 'era5_uwnd', 'era5_vwnd', 'displacement', 'heading']
buoy_data = buoy_data[columns_to_keep].copy()
buoy_data['datetime'] = pd.to_datetime(buoy_data['datetime'])

# Define features and targets
X = buoy_data[['Latitude', 'Longitude', 'era5_uwnd', 'era5_vwnd', 'BuoyID', 'datetime']]
y = buoy_data[['displacement', 'heading']]
groups = buoy_data['BuoyID']

# Models to evaluate
model_configs = [
    ('ElasticNet', MultiOutputRegressor(ElasticNet(alpha=1.0, l1_ratio=0.5))),
    ('GradientBoosting', MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, max_depth=5))),
    ('RandomForest', RandomForestRegressor(n_estimators=100, max_depth=10)),
    ('XGBoost', MultiOutputRegressor(XGBRegressor(n_estimators=100, max_depth=6, objective='reg:squarederror'))),
    ('LightGBM', MultiOutputRegressor(lgb.LGBMRegressor(n_estimators=100, max_depth=6, **lgb_params)))
]

# GroupKFold for cross-validation
cv_folds = 5
group_kf = GroupKFold(n_splits=cv_folds)

Hyperparameter tuning on the best model with Optuna

In [9]:
import optuna
from optuna.pruners import MedianPruner
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import lightgbm as lgb
import numpy as np
import gc

# Define the objective function for hyperparameter tuning
def objective(trial):
    print(f"Starting trial {trial.number}...")  # Track the start of each trial

    if best_model_row['Model'] == 'ElasticNet':
        alpha = trial.suggest_float('alpha', 0.1, 10.0, log=True)
        l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
        model = MultiOutputRegressor(ElasticNet(alpha=alpha, l1_ratio=l1_ratio))
    elif best_model_row['Model'] == 'GradientBoosting':
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        max_depth = trial.suggest_int('max_depth', 3, 10)
        model = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth))
    elif best_model_row['Model'] == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        max_depth = trial.suggest_int('max_depth', 5, 15)
        model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    elif best_model_row['Model'] == 'XGBoost':
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        max_depth = trial.suggest_int('max_depth', 3, 10)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        model = MultiOutputRegressor(XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, objective='reg:squarederror'))
    elif best_model_row['Model'] == 'LightGBM':
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        max_depth = trial.suggest_int('max_depth', 3, 10)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        model = MultiOutputRegressor(lgb.LGBMRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate))

    # Cross-validation logic
    model_scores = []
    for fold, (train_index, val_index) in enumerate(group_kf.split(X, y, groups=groups)):
        print(f"  Processing fold {fold + 1}...")  # Track folds
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Convert to numpy arrays and reduce precision
        X_train = X_train.drop(columns=['BuoyID', 'datetime']).to_numpy(dtype='float32')
        X_val = X_val.drop(columns=['BuoyID', 'datetime']).to_numpy(dtype='float32')
        y_train = y_train.to_numpy(dtype='float32')
        y_val = y_val.to_numpy(dtype='float32')

        # Train the model
        model.fit(X_train, y_train)

        # Iterative prediction
        y_pred = iterative_prediction(
            val_data=X.iloc[val_index],
            model=model,
            tree=tree,
            valid_times=valid_time_dt,
            latitudes=latitudes,
            longitudes=longitudes,
            lat_lon_pairs=lat_lon_pairs
        )

        # Filter out datetime column from predictions
        y_pred_filtered = y_pred[:, :2]  # Keep only Longitude and Latitude

        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(y_val, y_pred_filtered))
        model_scores.append(rmse)
        print(f"    Fold {fold + 1} RMSE: {rmse:.4f}")  # Track RMSE per fold

        # Free memory after each fold
        del X_train, X_val, y_train, y_val, y_pred, y_pred_filtered
        gc.collect()

    trial_score = np.mean(model_scores)
    print(f"Trial {trial.number} completed with mean RMSE: {trial_score:.4f}")  # Track trial completion
    return trial_score

# Create an Optuna study with pruning and enable parallel execution
study = optuna.create_study(direction='minimize', pruner=MedianPruner())

# Perform optimization with parallel processing
print("Starting hyperparameter tuning...")
study.optimize(objective, n_trials=10, n_jobs=-1)  # n_jobs=-1 utilizes all available CPU cores

# Retrieve the best parameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

[I 2024-11-28 13:24:58,750] A new study created in memory with name: no-name-df630455-a5f2-49ff-885a-a22e25ff1ce7


Starting hyperparameter tuning...
Starting trial 0...
Starting trial 1...
Starting trial 2...
Starting trial 3...
Starting trial 4...
Starting trial 5...
Starting trial 6...
Starting trial 7...
Starting trial 8...
Starting trial 9...
  Processing fold 1...
  Processing fold 1...
  Processing fold 1...  Processing fold 1...

  Processing fold 1...
  Processing fold 1...
  Processing fold 1...
  Processing fold 1...
  Processing fold 1...
  Processing fold 1...
    Fold 1 RMSE: 824.6651
  Processing fold 2...
    Fold 1 RMSE: 826.0157
  Processing fold 2...
    Fold 1 RMSE: 825.4650
  Processing fold 2...
    Fold 1 RMSE: 824.9197
  Processing fold 2...
    Fold 1 RMSE: 825.8214
  Processing fold 2...
    Fold 1 RMSE: 824.7591
  Processing fold 2...
    Fold 1 RMSE: 825.4667
  Processing fold 2...
    Fold 1 RMSE: 825.2453
  Processing fold 2...
    Fold 1 RMSE: 826.6707
  Processing fold 2...
    Fold 1 RMSE: 824.9205
  Processing fold 2...
    Fold 2 RMSE: 768.3914
  Processing fold 3.

[I 2024-11-29 02:12:03,779] Trial 4 finished with value: 1011.8157467065827 and parameters: {'n_estimators': 105, 'max_depth': 6, 'learning_rate': 0.08032856349779974}. Best is trial 4 with value: 1011.8157467065827.


    Fold 5 RMSE: 1830.6307
Trial 4 completed with mean RMSE: 1011.8157
    Fold 5 RMSE: 1830.6201


[I 2024-11-29 02:12:37,810] Trial 6 finished with value: 1013.1614311939305 and parameters: {'n_estimators': 81, 'max_depth': 5, 'learning_rate': 0.011056453615366158}. Best is trial 4 with value: 1011.8157467065827.


...

Trial 2 completed with mean RMSE: 1013.9424


[I 2024-11-29 02:38:57,873] Trial 7 finished with value: 1015.1000700608416 and parameters: {'n_estimators': 179, 'max_depth': 9, 'learning_rate': 0.03726743632388828}. Best is trial 9 with value: 1011.3034407694342.


    Fold 5 RMSE: 1831.6577
Trial 7 completed with mean RMSE: 1015.1001


[I 2024-11-29 02:39:13,806] Trial 1 finished with value: 1011.3993309347736 and parameters: {'n_estimators': 147, 'max_depth': 6, 'learning_rate': 0.07611857934040783}. Best is trial 9 with value: 1011.3034407694342.


    Fold 5 RMSE: 1829.1918
Trial 1 completed with mean RMSE: 1011.3993
Best parameters: {'n_estimators': 82, 'max_depth': 6, 'learning_rate': 0.2541334093676109}


Making predictions with the best tuned model and saving the results for evaluation

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
from geopy.distance import geodesic
import time

# Randomly select 5 buoys for validation
print("Selecting 5 random buoys for validation...")
np.random.seed(42)  # Set seed for reproducibility
validation_buoys = np.random.choice(X['BuoyID'].unique(), size=5, replace=False)
print(f"Selected validation buoys: {validation_buoys}")

# Split the data into training and validation based on BuoyID
print("Splitting data into training and validation sets...")
train_data = X[~X['BuoyID'].isin(validation_buoys)].copy()
val_data = X[X['BuoyID'].isin(validation_buoys)].copy()

# Ensure y is aligned with the indices of X
y_train = y.loc[train_data.index]
y_val = y.loc[val_data.index]

print(f"Training data size: {train_data.shape[0]} rows")
print(f"Validation data size: {val_data.shape[0]} rows")

# Drop 'BuoyID' and 'datetime' for training
print("Dropping unnecessary columns ('BuoyID', 'datetime') from training and validation sets...")
X_train_clean = train_data.drop(columns=['BuoyID', 'datetime'])
X_val_clean = val_data.drop(columns=['BuoyID', 'datetime'])

# Instantiate the model using best_params
print("Instantiating the model with the best parameters...")
if best_model_row['Model'] == 'ElasticNet':
    best_model = MultiOutputRegressor(ElasticNet(**best_params))
elif best_model_row['Model'] == 'GradientBoosting':
    best_model = MultiOutputRegressor(GradientBoostingRegressor(**best_params))
elif best_model_row['Model'] == 'RandomForest':
    best_model = RandomForestRegressor(**best_params)
elif best_model_row['Model'] == 'XGBoost':
    best_model = MultiOutputRegressor(XGBRegressor(**best_params, objective='reg:squarederror'))
elif best_model_row['Model'] == 'LightGBM':
    best_model = MultiOutputRegressor(lgb.LGBMRegressor(**best_params))

# Train the tuned model on the training data
print("Training the model on the training data...")
best_model.fit(X_train_clean, y_train)
print("Model training completed.")

# Use the iterative_prediction function for evaluation on validation buoys
print("Generating predictions for validation buoys...")
y_pred = iterative_prediction(
    val_data=val_data,
    model=best_model,
    tree=tree,
    valid_times=valid_time_dt,
    latitudes=latitudes,
    longitudes=longitudes,
    lat_lon_pairs=lat_lon_pairs
)
print("Predictions generated successfully.")

# Convert predictions to a DataFrame and include BuoyID and Datetime
print("Preparing predictions DataFrame...")
try:
    # Extract only latitude and longitude columns from y_pred
    pred_lat_lon = y_pred[:, :2]

    # Ensure the extracted data is a NumPy array of float type
    pred_lat_lon = np.array(pred_lat_lon, dtype=np.float64)

    # Round the latitude and longitude values to 3 decimal places
    pred_lat_lon = np.round(pred_lat_lon, 3)

    # Create the predictions DataFrame
    y_pred_df = pd.DataFrame(
        pred_lat_lon, columns=['Predicted Latitude', 'Predicted Longitude']
    )

    # Add metadata columns (BuoyID and Datetime)
    y_pred_df['BuoyID'] = val_data['BuoyID'].values
    y_pred_df['Datetime'] = val_data['datetime'].values

except Exception as e:
    print(f"Error during DataFrame creation: {e}")
    print(f"y_pred shape: {y_pred.shape}, y_pred content (first rows): {y_pred[:5]}")
    raise

# Calculate evaluation metrics
print("Calculating evaluation metrics...")

# Ensure valid arrays for true and predicted values
true_lat_lon = np.array(val_data[['Latitude', 'Longitude']].values, dtype=np.float64)
pred_lat_lon = np.array(y_pred_df[['Predicted Latitude', 'Predicted Longitude']].values, dtype=np.float64)

# Safeguard against scalar values
if true_lat_lon.ndim != 2 or pred_lat_lon.ndim != 2:
    raise ValueError(f"Expected 2D arrays for latitude/longitude, got shapes: "
                     f"true_lat_lon: {true_lat_lon.shape}, pred_lat_lon: {pred_lat_lon.shape}")

# Calculate metrics
lat_lon_rmse = np.sqrt(mean_squared_error(true_lat_lon, pred_lat_lon))
lat_lon_mae = mean_absolute_error(true_lat_lon, pred_lat_lon)
lat_lon_median_ae = median_absolute_error(true_lat_lon, pred_lat_lon)

# Haversine Distance
haversine_distances = [
    geodesic(true, pred).meters for true, pred in zip(true_lat_lon, pred_lat_lon)
]
mean_haversine_distance = np.mean(haversine_distances)
median_haversine_distance = np.median(haversine_distances)

print(f"Validation Latitude/Longitude RMSE: {lat_lon_rmse:.3f}")
print(f"Validation Latitude/Longitude MAE: {lat_lon_mae:.3f}")
print(f"Validation Latitude/Longitude Median AE: {lat_lon_median_ae:.3f}")
print(f"Mean Haversine Distance: {mean_haversine_distance:.3f} meters")
print(f"Median Haversine Distance: {median_haversine_distance:.3f} meters")

# Save predictions for analysis
print("Saving predictions to a CSV file...")
predictions_df = pd.DataFrame({
    'BuoyID': val_data['BuoyID'].values,
    'Datetime': val_data['datetime'].values,
    'True Latitude': np.round(val_data['Latitude'].values, 3),
    'True Longitude': np.round(val_data['Longitude'].values, 3),
    'Predicted Latitude': y_pred_df['Predicted Latitude'].values,
    'Predicted Longitude': y_pred_df['Predicted Longitude'].values,
    'Haversine Distance (m)': haversine_distances  # Include Haversine distance for each prediction
})

# Ensure the output directory exists
predictions_file = '../data/processed/predictions/bestmodel_predictions.csv'
os.makedirs(os.path.dirname(predictions_file), exist_ok=True)
predictions_df.to_csv(predictions_file, index=False)
print(f"Predictions saved successfully to: {predictions_file}")

Selecting 5 random buoys for validation...
Selected validation buoys: [300534063058450          900126 300234067977270 300534063050430
 300434066254600]
Splitting data into training and validation sets...
Training data size: 1568182 rows
Validation data size: 30424 rows
Dropping unnecessary columns ('BuoyID', 'datetime') from training and validation sets...
Instantiating the model with the best parameters...
Training the model on the training data...
Model training completed.
Generating predictions for validation buoys...
Predictions generated successfully.
Preparing predictions DataFrame...
Calculating evaluation metrics...
Validation Latitude/Longitude RMSE: 48.552
Validation Latitude/Longitude MAE: 28.511
Validation Latitude/Longitude Median AE: 18.578
Mean Haversine Distance: 1446983.693 meters
Median Haversine Distance: 1216817.940 meters
Saving predictions to a CSV file...
Predictions saved successfully to: ../data/processed/predictions/validation_predictions_lat_lon.csv
